In [1]:
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

In [2]:
model = read_sbml_model('models/iMM904_progesterone_fixed.xml') # Load model


#model.medium #see composition of growth medium
#model.metabolites.progesterone_c.summary
#model.reactions.R02216.summary
#model.metabolites.progesterone_c.summary(fva=0.95)
#model.metabolites.get_by_id("progesterone_c").upper_bound = 1000
#model.reactions.EX_glc__D_e
#model.reactions.EX_glc__D_e.flux_expression
#model.optimize()
#model.reactions.EX_glc__D_e.flux


In [3]:
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand") # Because it is a reversible reaction, we need to set a demand for production of progesterone 

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x07f50a0c293a0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


# Maximum theoretical productivity and yield of progesterone on glucose 

In [4]:
with model: 
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone
    print('Maximum theoretical productivity of progesterone:', progesterone_production, 'mmol/gDW*h') # Print max productivity
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # Define D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('Maximum theoretical yield of progesterone on glucose:', max_yield, 'mmol progesterone/mmol glucose') # Print max yield on glucose

Maximum theoretical productivity of progesterone: 0.14285714285714285 mmol/gDW*h
Maximum theoretical yield of progesterone on glucose: 0.0234375 mmol progesterone/mmol glucose


## Maximum theoretical productivity and yield in Cmole

In [37]:
# Define conversion factors
Cmole_glucose = 6
Cmole_progesterone = 21

In [ ]:
progesterone_production_Cmole = (progesterone_production)/(Cmole_progesterone) # Max productivity in Cmole
max_yield_Cmole = (max_yield*Cmole_progesterone)/(Cmole_glucose) # Max yield in Cmole
print('Maximum theoretical productivity of progesterone:', progesterone_production_Cmole, 'mmol/gDW*h') # Print max productivity in Cmole
print('Maximum theoretical yield of progesterone on glucose:', max_yield_Cmole, 'mmol progesterone/mmol glucose') # Print max yield on glucose in Cmole

# Increasing nutrient availability in the growth medium

In [38]:
# See medium compositions
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

Glucose availability in the medium is rather low - therefore it could be interesting to increase the amount of glucose in the medium and first see if it affects the theoretical growth rate.

In [ ]:
# Define D-glucose exchange reaction
D_glucose_exchange = model.reactions.get_by_id('EX_glc__D_e')

In [ ]:
# Modify the medium composition and simulate
medium = model.medium
with model:
    growth = model.optimize().objective_value # Simulate model
    print('At start glucose concentration:'.format(medium[D_glucose_exchange.id]), 'the theoretical maximum growth rate is:', growth, 'h^-1' )
    medium[D_glucose_exchange.id] = 1000 # Set D-glucose to max (1000)
    model.medium = medium
    growth = model.optimize().objective_value
    print('At new glucose concentration:'.format(medium[D_glucose_exchange.id]), 'the theoretical maximum growth rate is:', growth, 'h^-1' )

The growth rate is xxxxx, so it could be interesting to see the effect of increasing glucose in the medium on progesterone production.

## The effect on progesterone production by increasing glucose availability

In [ ]:
medium = model.medium
with model: 
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # Define D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('At start glucose concentration:', format(medium[D_glucose_exchange.id]), 'the theoretical maximum productivity of progesterone is:', progesterone_production, 'mmol/gDW*h and the maximum theoretical yield is:', max_yield, 'mmol progesterone/mmol glucose')
    
   
    medium[D_glucose_exchange.id] = 1000 # Set D-glucose to max
    model.medium = medium 

    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone and new glucose flux
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('At new glucose concentration:', format(medium[D_glucose_exchange.id]), 'the theoretical maximum productivity of progesterone is:', progesterone_production, 'mmol/gDW*h and the maximum theoretical yield is:', max_yield, 'mmol progesterone/mmol glucose')

Comparison with litterature